In [16]:
import pandas as pd
import numpy as np

class AHP:
    #传入的np.ndarray是的判断矩阵
    def __init__(self,array):
        self.array = array
        # 记录矩阵大小
        self.n = array.shape[0]
        # 初始化RI值,用于一致性检验 
        RI_list = [0,0,0.58,0.90,1.12,1.24,1.32,1.41,1.45]
        self.RI = RI_list[self.n-1]

        eig_val ,eig_vector = np.linalg.eig(self.array)
        #获取最大特征值
        self.max_val = np.max(eig_val)
        
	#获取最大特征值和对应的特征向量
    def get_eig(self):
        #numpy.linalg.eig() 计算矩阵特征值与特征向量
        eig_val ,eig_vector = np.linalg.eig(self.array)
        #获取最大特征值
        max_val = np.max(eig_val)
        max_val = round(max_val.real, 4)
        #通过位置来确定最大特征值对应的特征向量
        index = np.argmax(eig_val)
        max_vector = eig_vector[:,index]
        max_vector = max_vector.real.round(4) 
        #添加最大特征值属性
        self.max_val = max_val
        #计算权重向量W
        weight_vector = max_vector/sum(max_vector)
        weight_vector = weight_vector.round(4)
        #打印结果
        # print("最大的特征值: "+str(max_val))
        # print("对应的特征向量为: "+str(max_vector))
        # print("归一化后得到权重向量: "+str(weight_vector))
        return weight_vector
    
    #测试一致性
    def test_consitst(self):
        #计算CI值
        CI = (self.max_val-self.n)/(self.n-1) 
        CI = np.round(CI,4) 
        #打印结果
        print("判断矩阵的CI值为" +str(CI))
        print("判断矩阵的RI值为" +str(self.RI))
        #分类讨论
        if self.n == 2:
            print("仅包含两个子因素，不存在一致性问题")
        else:
            #计算CR值
            CR = CI/self.RI 
            CR = np.round(CR,4)
            #CR < 0.10才能通过检验
            if  CR < 0.10 :
                print("判断矩阵的CR值为" +str(CR) + "，通过一致性检验")
                return True
            else:
                print("判断矩阵的CR值为" +str(CR) + "，未通过一致性检验")
                return False  

criteria = np.array([[1, 1, 1, 4, 1, 1/2 ],
                     [1, 1, 2, 4, 1, 1/2 ],
                     [1, 1/2, 1, 5, 3, 1/2 ],
                     [1/4, 1/4, 1/5, 1, 1/3, 1/3 ],
                     [1, 1, 1/3, 3, 1, 1 ],
                     [2, 2, 2, 3, 3, 1 ]])

weight = AHP(criteria).get_eig()
print('各个准则对应的权重分别为：',weight)
# print(AHP(criteria).test_consitst()==true)

各个准则对应的权重分别为： [0.1507 0.1792 0.1886 0.0472 0.1464 0.2879]


In [ ]:


#极小型指标 -> 极大型指标
def dataDirection_1(datas):         
		return np.max(datas)-datas    

#中间型指标 -> 极大型指标
def dataDirection_2(datas, x_best):
    temp_datas = datas - x_best
    M = np.max(abs(temp_datas))
    answer_datas = 1 - abs(datas - x_best) / M     
    return answer_datas
    
#区间型指标 -> 极大型指标
def dataDirection_3(datas, x_min, x_max):
    M = max(x_min - np.min(datas), np.max(datas) - x_max)
    answer_list = []
    for i in datas:
        if(i < x_min):
            answer_list.append(1 - (x_min-i) /M)     
        elif( x_min <= i <= x_max):
            answer_list.append(1)
        else:
            answer_list.append(1 - (i - x_max)/M)
    return np.array(answer_list)   

def topsis(data, weight=None):
	# 归一化
	data = data / np.sqrt((data ** 2).sum())

	# 最优最劣方案
	Z = pd.DataFrame([data.min(), data.max()], index=['负理想解', '正理想解'])

	# 距离
	weight = entropyWeight(data) if weight is None else np.array(weight)
	Result = data.copy()
	Result['正理想解'] = np.sqrt(((data - Z.loc['正理想解']) ** 2 * weight).sum(axis=1))
	Result['负理想解'] = np.sqrt(((data - Z.loc['负理想解']) ** 2 * weight).sum(axis=1))

	# 综合得分指数
	Result['综合得分指数'] = Result['负理想解'] / (Result['负理想解'] + Result['正理想解'])
	Result['排序'] = Result.rank(ascending=False)['综合得分指数']

	return Result, Z, weight

data = pd.DataFrame(
    {'信道能耗成本': [0.1, 0.2, 0.4, 0.9, 1.2], '无线介质可用性': [5, 6, 7, 10, 2], '网络拥塞': [5000, 6000, 7000, 10000, 400],
     '成本': [4.7, 5.6, 6.7, 2.3, 1.8],'能耗': [4.7, 5.6, 6.7, 2.3, 1.8],'成本': [4.7, 5.6, 6.7, 2.3, 1.8]}, index=['cell network','wifi','bluetooth'])

data['生师比'] = dataDirection_3(data['生师比'], 5, 6, 2, 12)   # 师生比数据为区间型指标
data['逾期毕业率'] = 1 / data['逾期毕业率']   # 逾期毕业率为极小型指标

out = topsis(data, weight=[0.2, 0.3, 0.4, 0.1])    # 设置权系数